### Transform

In [9]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
%matplotlib inline  

In [10]:
rotation_range = 10
zoom_range = 0.05
shift_range = 0.05

In [12]:
image = cv2.imread("./img/cage.jpg", cv2.COLOR_RGB2BGR).astype(np.float32)
image = cv2.resize(image, (256, 256))

In [13]:
h, w = image.shape[0:2]

In [14]:
rotation = np.random.uniform(-rotation_range, rotation_range)
scale = np.random.uniform(1 - zoom_range, 1 + zoom_range)
tx = np.random.uniform(-shift_range, shift_range) * w
ty = np.random.uniform(-shift_range, shift_range) * h

In [15]:
mat = cv2.getRotationMatrix2D((w // 2, h // 2), rotation, scale)
mat[:, 2] += (tx, ty)

In [16]:
mat

array([[  1.00269863e+00,  -5.64824293e-03,  -6.03917026e+00],
       [  5.64824293e-03,   1.00269863e+00,   5.47119381e+00]])

In [ ]:
result = cv2.warpAffine(image, mat, (w, h), borderMode=cv2.BORDER_REPLICATE)

In [70]:
#result = result.transpose((1, 2, 0))
cv2.imwrite("./cage2.jpg", result)

True

### Warp

In [71]:
coverage = 160
scale = 5
zoom = 1

In [75]:
## License (Modified BSD)
## Copyright (C) 2011, the scikit-image team All rights reserved.
##
## Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:
##
## Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.
## Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.
## Neither the name of skimage nor the names of its contributors may be used to endorse or promote products derived from this software without specific prior written permission.
## THIS SOFTWARE IS PROVIDED BY THE AUTHOR ``AS IS'' AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE AUTHOR BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.

# umeyama function from scikit-image/skimage/transform/_geometric.py

def umeyama(src, dst, estimate_scale):
    """Estimate N-D similarity transformation with or without scaling.
    Parameters
    ----------
    src : (M, N) array
        Source coordinates.
    dst : (M, N) array
        Destination coordinates.
    estimate_scale : bool
        Whether to estimate scaling factor.
    Returns
    -------
    T : (N + 1, N + 1)
        The homogeneous similarity transformation matrix. The matrix contains
        NaN values only if the problem is not well-conditioned.
    References
    ----------
    .. [1] "Least-squares estimation of transformation parameters between two
            point patterns", Shinji Umeyama, PAMI 1991, DOI: 10.1109/34.88573
    """

    num = src.shape[0]
    dim = src.shape[1]

    # Compute mean of src and dst.
    src_mean = src.mean(axis=0)
    dst_mean = dst.mean(axis=0)

    # Subtract mean from src and dst.
    src_demean = src - src_mean
    dst_demean = dst - dst_mean

    # Eq. (38).
    A = np.dot(dst_demean.T, src_demean) / num

    # Eq. (39).
    d = np.ones((dim,), dtype=np.double)
    if np.linalg.det(A) < 0:
        d[dim - 1] = -1

    T = np.eye(dim + 1, dtype=np.double)

    U, S, V = np.linalg.svd(A)

    # Eq. (40) and (43).
    rank = np.linalg.matrix_rank(A)
    if rank == 0:
        return np.nan * T
    elif rank == dim - 1:
        if np.linalg.det(U) * np.linalg.det(V) > 0:
            T[:dim, :dim] = np.dot(U, V)
        else:
            s = d[dim - 1]
            d[dim - 1] = -1
            T[:dim, :dim] = np.dot(U, np.dot(np.diag(d), V))
            d[dim - 1] = s
    else:
        T[:dim, :dim] = np.dot(U, np.dot(np.diag(d), V.T))

    if estimate_scale:
        # Eq. (41) and (42).
        scale = 1.0 / src_demean.var(axis=0).sum() * np.dot(S, d)
    else:
        scale = 1.0

    T[:dim, dim] = dst_mean - scale * np.dot(T[:dim, :dim], src_mean.T)
    T[:dim, :dim] *= scale

    return T

In [76]:
range_ = np.linspace(128 - coverage//2, 128 + coverage//2, 5)
mapx = np.broadcast_to(range_, (5, 5))
mapy = mapx.T

mapx = mapx + np.random.normal(size=(5,5), scale=scale)
mapy = mapy + np.random.normal(size=(5,5), scale=scale)

interp_mapx = cv2.resize(mapx, (80*zoom,80*zoom))[8*zoom:72*zoom,8*zoom:72*zoom].astype('float32')
interp_mapy = cv2.resize(mapy, (80*zoom,80*zoom))[8*zoom:72*zoom,8*zoom:72*zoom].astype('float32')

warped_image = cv2.remap(image, interp_mapx, interp_mapy, cv2.INTER_LINEAR)

src_points = np.stack([mapx.ravel(), mapy.ravel() ], axis=-1)
dst_points = np.mgrid[0:65*zoom:16*zoom,0:65*zoom:16*zoom].T.reshape(-1,2)
mat = umeyama(src_points, dst_points, True)[0:2]

target_image = cv2.warpAffine(image, mat, (64*zoom,64*zoom))

In [77]:
cv2.imwrite("./cage3.jpg", warped_image)

True

## Hue, Saturation & Brightness

In [72]:
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
h, s, v = cv2.split(hsv)
h += np.random.uniform(-7, 7)
hsv = cv2.merge((h, s, v))
img = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
#img = img[:, ::-1]
cv2.imwrite("./cage6.jpg", img)

True

In [ ]:
if numpy.random.random() < random_flip:
            result = result[:, ::-1]